# 🎤 Preparazione Dataset per Piper TTS

Questo notebook prepara il dataset LJSpeech-Italian per training con Piper TTS.

## 📊 Dataset Disponibili:

### 1. z-uo/female-LJSpeech-italian (CONSIGLIATO per call center)
- 🎤 Voce femminile
- ⏱️ Durata: 8h 23m
- 🔊 Sample rate: 16kHz
- 📦 ~600 MB

### 2. z-uo/male-LJSpeech-italian
- 🎤 Voce maschile
- ⏱️ Durata: 31h 45m
- 🔊 Sample rate: 16kHz
- 📦 ~2.5 GB

## ⚙️ Setup:
1. Runtime → Change runtime type → **GPU T4**
2. Monta Google Drive
3. Scegli dataset (female/male) nella cella di download
4. Esegui celle in ordine

## 1️⃣ Setup Ambiente

In [ ]:
# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Installa dipendenze
!pip install -q datasets soundfile tqdm torchcodec
print("✅ Installazione completata!")

## 2️⃣ Download LJSpeech-Italian

In [ ]:
# ============================================================
#  DOWNLOAD LJSPEECH-IT - VERSIONE CORRETTA
# ============================================================

print("="*60)
print("  DOWNLOAD LJSPEECH-IT")
print("="*60)
print()

import os
from datasets import load_dataset
from tqdm import tqdm
import soundfile as sf

# Installa torchcodec se mancante
try:
    import torchcodec
except ImportError:
    print("📦 Installazione torchcodec...")
    !pip install -q torchcodec
    print("✅ torchcodec installato")

# Scegli dataset (female o male)
DATASET_NAME = "z-uo/female-LJSpeech-italian"  # 8h 23m, voce femminile
# DATASET_NAME = "z-uo/male-LJSpeech-italian"  # 31h 45m, voce maschile (commenta la riga sopra e decommenta questa)

print(f"📥 Scaricamento dataset: {DATASET_NAME}...")
dataset = load_dataset(DATASET_NAME, split="train")
print(f"✅ Dataset caricato: {len(dataset)} campioni")

# Crea directory su Drive
dataset_dir = "/content/drive/MyDrive/piper_training/dataset/ljspeech_italian"
wavs_dir = os.path.join(dataset_dir, "wavs")
os.makedirs(wavs_dir, exist_ok=True)

print(f"\n💾 Salvataggio su Google Drive...")
print(f"📁 Percorso: {dataset_dir}")

# Prepara metadata
metadata_lines = []
total = len(dataset)

for idx in tqdm(range(total), desc="Salvando audio"):
    item = dataset[idx]
    
    # Salva audio
    audio_filename = f"LJ_{idx:06d}.wav"
    audio_path = os.path.join(wavs_dir, audio_filename)
    
    # Estrai audio array e sample rate
    audio_data = item['audio']
    sf.write(audio_path, audio_data['array'], audio_data['sampling_rate'])
    
    # Aggiungi a metadata
    text = item['text']  # Nota: 'text' non 'sentence'
    metadata_lines.append(f"{audio_filename}|{text}")

# Salva metadata.csv
metadata_path = os.path.join(dataset_dir, "metadata.csv")
with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"\n✅ Completato!")
print(f"   📊 {total} file audio salvati")
print(f"   📄 metadata.csv creato")
print(f"   💾 Totale: ~{total * 0.5:.1f} MB")
print(f"\n📋 Dataset usato: {DATASET_NAME}")
print(f"   🎤 Tipo: {'Voce femminile' if 'female' in DATASET_NAME else 'Voce maschile'}")

## 3️⃣ Verifica Dataset

In [ ]:
# Verifica file creati
import pandas as pd
from IPython.display import Audio, display

dataset_dir = "/content/drive/MyDrive/piper_training/dataset/ljspeech_italian"
metadata_path = os.path.join(dataset_dir, "metadata.csv")

# Carica metadata
df = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
print(f"📊 Dataset info:")
print(f"   File audio: {len(df)}")
print(f"   Testo totale: {df['text'].str.len().sum()} caratteri")
print(f"   Testo medio: {df['text'].str.len().mean():.1f} caratteri/sample")

# Test audio casuale
sample_idx = 0
sample_row = df.iloc[sample_idx]
sample_audio = os.path.join(dataset_dir, "wavs", sample_row['filename'])

print(f"\n🎤 Sample test #{sample_idx}:")
print(f"   Testo: {sample_row['text']}")
print(f"   File: {sample_row['filename']}")

display(Audio(sample_audio))

## 4️⃣ Statistiche Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Lunghezza testi
text_lengths = df['text'].str.len()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(text_lengths, bins=50, edgecolor='black')
plt.xlabel('Lunghezza testo (caratteri)')
plt.ylabel('Frequenza')
plt.title('Distribuzione lunghezza testi')
plt.axvline(text_lengths.mean(), color='r', linestyle='--', label=f'Media: {text_lengths.mean():.1f}')
plt.legend()

plt.subplot(1, 2, 2)
word_counts = df['text'].str.split().str.len()
plt.hist(word_counts, bins=50, edgecolor='black')
plt.xlabel('Numero parole')
plt.ylabel('Frequenza')
plt.title('Distribuzione numero parole')
plt.axvline(word_counts.mean(), color='r', linestyle='--', label=f'Media: {word_counts.mean():.1f}')
plt.legend()

plt.tight_layout()
plt.show()

print("📈 Statistiche:")
print(f"   Lunghezza testo: {text_lengths.min()}-{text_lengths.max()} caratteri")
print(f"   Parole per sample: {word_counts.min()}-{word_counts.max()} parole")
print(f"   Totale parole: {word_counts.sum():.0f}")

## ✅ Dataset Pronto!

Il dataset è ora pronto per training con Piper TTS.

### Prossimi passi:
1. Segui la guida ufficiale Piper: https://github.com/rhasspy/piper
2. Configura training con il dataset preparato
3. Avvia training (richiede diverse ore su GPU)

### Note:
- Dataset: `/content/drive/MyDrive/piper_training/dataset/ljspeech_italian`
- Metadata: `metadata.csv` (formato: filename|text)
- Audio: cartella `wavs/` (formato WAV)